In [2]:
from typing import Dict, Optional
from datasets import Dataset, load_dataset, concatenate_datasets
import os
import glob
import json
import copy
import threading
import time
import random
from typing import Dict, Optional
import requests
import numpy as np
import re

import fasttext
lang_detect = fasttext.load_model('../fastchat/modules/fasttext/lid.176.bin')

from fastchat.modules.answer_refiner import generate_refiner

import chromadb
from chromadb.config import Settings
import random
from fastchat.modules.embedder_adapter import Embedder, get_embedder
from fastchat.conversation import (
    SeparatorStyle,
)
from fastchat.model.model_adapter import get_conversation_template
import copy
from fastchat.train.data_modules.sft_dataset import load_sft_dataset, combine_dataset
from fastchat.train.data_modules.dpo_dataset import load_dpo_dataset
from fastchat.train.data_modules.dedup import (
    dedup_by_similarity,
    dedup_non_pair,
    dedup_repetition,
    dedup_math,
    dedup_too_much_token,
    dedup_short,
)

In [ ]:
# def dedup_by_similarity(dataset, prompt_template='chat-orca', target_text_len=100, n_results=100, distance_threshold = 0.6):
_dataset = train_dataset
prompt_template='vicuna'
target_text_len=100
n_results=100
distance_threshold = 0.35
    
if prompt_template == 'chat-orca':
    conv = get_conversation_template(prompt_template)
    system_message = conv.system_message
    sep_style = conv.sep_style
    sep = conv.sep
    prompt_user, prompt_bot = conv.roles

    len_sep_style = 0
    if sep_style == SeparatorStyle.ADD_COLON_TWO:
        len_sep_style = 1

    len_front = len(system_message) + len(sep) + len(prompt_user) + len_sep_style + 1
    len_rear = len(sep) + len(prompt_bot) + len_sep_style
    def filter_question(data):
        return { 
            # **data,
            'prompt': data['prompt'][len_front:-len_rear][:target_text_len]
        }

if prompt_template == 'vicuna':
    def filter_question(data):
        return {
            'prompt': data['conversations'][0]['value'][:target_text_len]
        }

question_dataset = _dataset.map(filter_question)

chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
collection = chroma_client.create_collection(name="context", embedding_function=embedder.embed, metadata={"hnsw:space": "cosine"})
ids = []
# add
texts = question_dataset['prompt']
last_id = -1
new_ids = [f"id{i+last_id+1}" for i in range(len(texts))]
ids += new_ids
collection.add(documents=texts, ids=new_ids)

query_ids = copy.deepcopy(new_ids)
selected_ids = []
duplicated_ids = []

weird_ids = []
error_ids = []
while query_ids:
    current_id = random.choice(query_ids)
    if current_id in selected_ids:
        print("Warning: this is weird..")
        weird_ids.append(current_id)
        continue
    selected_ids.append(current_id)
    search_strings = [texts[int(current_id[2:])]]
    if collection.count() == 0:
        print("Warning: collection is empty. Forced break")
        break
    result = collection.query(query_texts=search_strings, n_results=min(n_results, len(query_ids)), include=['distances']) #'documents'

    search_ids = result['ids'][0]
    distances = result['distances'][0]
    remove_ids = []
    for idx in range(len(search_ids)):
        sid = search_ids[idx]
        dist = distances[idx]
        if dist < distance_threshold:
            remove_ids.append(sid)

    for rid in remove_ids:
        if rid in query_ids:
            query_ids.remove(rid)
            
    if remove_ids:
        duplicated_ids += remove_ids
        collection.delete(ids=remove_ids)
    else:
        print("Warning: this is error..")
        error_ids.append(current_id)

    print(f"Total:{len(new_ids)} Selected:{len(selected_ids)} current_dup:{len(remove_ids)} vector_store:{collection.count()} remained:{len(query_ids)} total_dup:{len(duplicated_ids)}", '\t\t\t\t\t', end='\r')

print('finished dedup data:', f"Total:{len(new_ids)} Selected:{len(selected_ids)} current_dup:{len(remove_ids)} vector_store:{collection.count()} remained:{len(query_ids)} total_dup:{len(duplicated_ids)}")

selected_ids = [int(sid[2:]) for sid in set(selected_ids)]

_dataset = _dataset.select(selected_ids)

# return dataset, selected_ids, query_ids

In [ ]:
qna_list = [
    "/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json",
    "/data/llm_datasets/custom/vicuna_format/koalpaca_v1.1-vicuna.json",
    "/data/llm_datasets/custom/deduped/alpaca-gpt4-korean_dedup/",
    "/data/llm_datasets/custom/vicuna_format/korquad-chat-vicuna.json",
    "/data/llm_datasets/custom/vicuna_format/wizardlm_orca_vicuna.json",
    "/data/llm_datasets/sharegpt_gpt4/sharegpt_gpt4.jsonl",
    "/data/llm_datasets/custom/vicuna_format/sharegpt_V3_format_others.json",
    "/data/llm_datasets/custom/deduped/sharegpt_V3_format_ko_selected.json",
    "/data/llm_datasets/custom/vicuna_format/lima_vicuna_format_ko.json",
]

correction_list = [
    "/data/llm_datasets/custom/vicuna_format/KoreaSpellingCorrection/"
]

summary_list = [
    "/data/llm_datasets/custom/deduped/aihub_summary_data_tech_dedup/",
    "/data/llm_datasets/aihub_summary_data/도서/",
    "/data/llm_datasets/aihub_summary_data/법률/",
    "/data/llm_datasets/custom/deduped/naver-news-summarization-ko-vicuna_dedup/",
    
]

translation_list = [
    "/data/llm_datasets/custom/vicuna_format/sharegpt_V3_format_translation(enko).json",
    "/data/llm_datasets/custom/vicuna_format/sharegpt_V3_format_translation(koen).json",
]


dataset_list = qna_list + correction_list + summary_list + translation_list

In [ ]:
# dedup2
qna_list = [
    "/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json",
    "/data/llm_datasets/custom/vicuna_format/koalpaca_v1.1-vicuna.json",
    "/data/llm_datasets/custom/deduped2/alpaca-gpt4-korean_dedup2.json",
    "/data/llm_datasets/custom/vicuna_format/korquad-chat-vicuna.json",
    "/data/llm_datasets/custom/deduped2/wizardlm_orca_vicuna_dedup2.json",
    "/data/llm_datasets/sharegpt_gpt4/sharegpt_gpt4.jsonl",#
    "/data/llm_datasets/custom/vicuna_format/sharegpt_V3_format_others.json",#
    "/data/llm_datasets/custom/deduped2/sharegpt_V3_format_ko_selected_dedup2.json",
    "/data/llm_datasets/custom/deduped2/lima_vicuna_format_ko.json",
]

# correction_list = [
#     "/data/llm_datasets/custom/deduped2/KoreaSpellingCorrection-10000.json",
# ]

summary_list = [
    "/data/llm_datasets/custom/deduped2/aihub_summary_data_tech_dedup-5000.json",
    "/data/llm_datasets/custom/deduped2/aihub_summary_data_book-5000.json",
    "/data/llm_datasets/custom/deduped2/aihub_summary_data_law-5000.json",
    "/data/llm_datasets/custom/deduped2/naver-news-summarization-ko-vicuna_dedup-5000.json",
    
]

translation_list = [
    "/data/llm_datasets/custom/deduped2/sharegpt_V3_format_translation(enko)-10000.json",
    "/data/llm_datasets/custom/deduped2/sharegpt_V3_format_translation(koen)-10000.json",
]


dataset_list = qna_list + summary_list + translation_list

In [107]:
# refine
qna_list = [
    "/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json",
    "/data/llm_datasets/custom/vicuna_format/koalpaca_v1.1-vicuna.json",
    "/data/llm_datasets/custom/refined/alpaca-gpt4-korean_dedup2.json",
    "/data/llm_datasets/custom/vicuna_format/korquad-chat-vicuna.json",
    "/data/llm_datasets/custom/refined/wizardlm_orca_vicuna_dedup2.json",
    "/data/llm_datasets/custom/vicuna_format/sharegpt_gpt4.json",#
    "/data/llm_datasets/custom/vicuna_format/sharegpt_V3_format_others.json",#
    "/data/llm_datasets/custom/refined/sharegpt_V3_format_ko_selected_dedup2.json",
    "/data/llm_datasets/custom/refined/lima_vicuna_format_ko.json",
]

# correction_list = [
#     "/data/llm_datasets/custom/deduped2/KoreaSpellingCorrection-10000.json",
# ]

summary_list = [
    "/data/llm_datasets/custom/deduped2/aihub_summary_data_tech_dedup-5000.json",
    "/data/llm_datasets/custom/deduped2/aihub_summary_data_book-5000.json",
    "/data/llm_datasets/custom/deduped2/aihub_summary_data_law-5000.json",
    "/data/llm_datasets/custom/deduped2/naver-news-summarization-ko-vicuna_dedup-5000.json",
    
]

translation_list = [
    "/data/llm_datasets/custom/deduped2/sharegpt_V3_format_translation(enko)-10000.json",
    "/data/llm_datasets/custom/deduped2/sharegpt_V3_format_translation(koen)-10000.json",
]


dataset_list = qna_list + summary_list + translation_list

In [167]:
# dpo v2
dpo_list = [
    "/data/llm_datasets/ultrafeedback_binarized/data/train_prefs-00000-of-00001-17309c769bfe5733.parquet",
    "/data/llm_datasets/orca_dpo_pairs/",
    "/data/llm_datasets/distilabel-math-preference-dpo/data/",
]

dpo_list2 = [
    "/data/llm_datasets/custom/kodpo/untranslated/ultrafeedback_binarized.json",
    "/data/llm_datasets/custom/kodpo/untranslated/orca_dpo_pairs.json",
    "/data/llm_datasets/custom/kodpo/untranslated/distilabel-math-preference-dpo.json",
]

dpo_list3 = [
    "/data/llm_datasets/custom/kodpo/translated/ko_ultrafeedback_binarized.json",
    "/data/llm_datasets/custom/kodpo/translated/ko_orca_dpo_pairs.json",
    "/data/llm_datasets/custom/kodpo/translated/ko_distilabel-math-preference-dpo.json",
]

# dedup
dpo_list4 = [
    '/data/llm_datasets/custom/kodpo/deduped/ko_ultrafeedback_binarized.json',
    "/data/llm_datasets/custom/kodpo/translated/ko_orca_dpo_pairs.json",
    "/data/llm_datasets/custom/kodpo/translated/ko_distilabel-math-preference-dpo.json",
]

# refine codeblock
dpo_list5 = [
    '/data/llm_datasets/custom/kodpo/refined/ko_ultrafeedback_binarized.json',
    "/data/llm_datasets/custom/kodpo/translated/ko_orca_dpo_pairs.json",
    "/data/llm_datasets/custom/kodpo/translated/ko_distilabel-math-preference-dpo.json",
]

In [189]:
dataset = load_dpo_dataset(dpo_list5[2])

In [ ]:
dataset[2000]

In [190]:
mdpo.append(random_select(dataset, 1000))

In [ ]:
new_dataset_list = []
for d in dataset_list:
    new_dataset_list.append("\"" + d + "\"")
print(' '.join(new_dataset_list))

In [184]:
mdpo = []

In [133]:
msft = []

In [158]:
dataset_path = dataset_list[14]
print(dataset_path)
dataset = load_sft_dataset(dataset_path)
dataset

/data/llm_datasets/custom/deduped2/sharegpt_V3_format_translation(koen)-10000.json


Dataset({
    features: ['id', 'conversations', 'task'],
    num_rows: 10000
})

In [159]:
msft.append(random_select(dataset, 50))

In [119]:
def random_select(dataset, num_select):
    return dataset.select(np.random.choice(list(range(len(dataset))), num_select))

In [191]:
combined_dataset = concatenate_datasets(mdpo)

In [161]:
combined_dataset = combined_dataset.shuffle(42)

In [164]:
combined_dataset[2]

{'id': 'naver-news-summarization-ko_9179',
 'conversations': [{'from': 'human',
   'value': '이기숙 부영그룹 고문 아이와의 대화 어떻게 할 것인가 주제로 강연 부영그룹이 자사가 지원하는 부영 사랑으로 어린이집 학부모를 대상으로 비대면 부모 교육을 실시했다. 부영그룹은 전국 65개원 부영 사랑으로 어린이집 학부모들을 대상으로 5일 간 온라인 화상 강의 방식의 비대면 교육을 진행했다고 4일 밝혔다. 학부모 약 1천여명이 온라인 화상 강의에 참석했다. 부영사옥 전경. 사진 부영 이번 교육은 국내 유아교육 권위자인 이기숙 부영그룹 고문 이화여대 명예교수 이 아이와의 대화 어떻게 할 것인가 를 주제로 언어적 대화와 비언어적 대화가 무엇인지에 대해 강연했다. 부영그룹은 지난 2016년부터 전국 65개 부영 아파트 단지 내 관리동 어린이집인 부영 사랑으로 어린이집 에 대해 임대료를 없애고 그 비용을 학부모 부담금 절감 등 어린이집을 이용하는 영유아들의 보육과 복지를 위해 사용하도록 하고 있다. 한편 부영그룹 보육지원팀은 유아교육전문가들로 구성돼 있으며 공정한 원장 선발과 함께 부모교육 교사교육 보육 컨설팅 각종 보육 행사 등을 무상으로 제공 보육의 질을 높이기 위해 노력하고 있다.'},
  {'from': 'gpt',
   'value': '지난 약 1천여명의 학부모가 참석한 부영그룹이 자사가 지원하는 부영 사랑으로 어린이집 학부모를 대상으로 5일 간 온라인 화상 강의 방식의 비대면 교육을 진행하고 아이와의 대화 어떻게 할 것인가 를 주제로 언어적 대화와 비언어적 대화가 무엇인지에 대해 강연했다.'}],
 'instruction': None,
 'task_name': None,
 'task': 'summarization'}

In [100]:
# from fastchat.train.data_modules.sft_dataset import load_sft_dataset, combine_dataset
combined_dataset = concatenate_datasets([load_sft_dataset(dataset_path) for dataset_path in dataset_list])
combined_dataset.features

{'id': Value(dtype='string', id=None),
 'conversations': [{'from': Value(dtype='string', id=None),
   'value': Value(dtype='string', id=None)}],
 'instruction': Value(dtype='string', id=None),
 'task_name': Value(dtype='string', id=None),
 'system': Value(dtype='string', id=None),
 'task': Value(dtype='string', id=None)}

In [192]:
new_dataset = []
for data in combined_dataset:
    new_dataset.append(data)    


In [193]:
# combined_dataset.to_json("/data/llm_datasets/custom/ados_sft_v4.json")
with open("/data/llm_datasets/custom/ados/dpo/ados_mdpo_v2.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [194]:
dpo_dataset = ados_DPODataset("/data/llm_datasets/custom/ados/dpo/ados_mdpo_v2.json")

In [196]:
dpo_datamodule = dpo_dataset.make_dpo_data_module()

Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1349.52it/s]
Generating train split: 11000 examples [00:01, 8384.35 examples/s]
Map: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11000/11000 [00:01<00:00, 9377.51 examples/s]


In [200]:
dpo_datamodule['train_dataset'][0]

{'chosen': ' 프랑스어로 제공된 제품 리뷰에 따르면 리뷰의 감성은 부정적입니다. 사용자는 책이 너무 단순하고 부모를 위한 것인지 어린이를 위한 것인지 불분명하다고 언급하고 있습니다. 또한 어린이가 쓴 목소리와 말로 쓰여져 있어 어린이는 읽을 수 없고 어른은 유치하다고 비판하고 있습니다. 따라서 리뷰의 전반적인 감성은 부정적입니다. 따라서 리뷰는 "NEG"로 분류할 수 있습니다.</s>',
 'rejected': ' NEG\n신뢰도: 85%</s>',
 'prompt': '### System:\nYou are an AI assistant, please behave and help the user.\n\n### User: 자세한 지침: 이 과제에서는 프랑스어로 된 책 리뷰가 주어집니다. 리뷰의 전체적인 감정이 긍정적이면 "POS"로 분류하고, 리뷰의 전체적인 감정이 부정적이면 "NEG"로 분류하는 것이 목표입니다.\nQ: 너무 간단해요 . 이 책은 너무 단순합니다. 부모를 대상으로 한 것인지 아이들을 대상으로 한 것인지 잘 모르겠어요 (제 아이는 더 잘 알아요). 이 책은 어린 아이의 목소리와 언어로 쓰여져 있습니다. 하지만 아이들은 읽을 수 없어요 !!! 그리고 어른들은 이 책이 너무 유치하다고 생각합니다. 결론적으로, 적용되지 않은 좋은 아이디어가 가득합니다.\nA:\n\n### Assistant:'}

In [136]:
""" find odd code blocks"""
dataset_path = "/data/llm_datasets/custom/ados_sft_v4.json"
# dataset = load_dataset("json", dataset_path)
dataset = load_sft_dataset(dataset_path, split=None)
train_dataset = dataset['train']

# new_dataset = []
code_prefixes = []

odd_dataset = []
oddd_dataset = []
odd_idxs = set()
normal_dataset = []
flag_normal = True
flag_code = False
for idx, data in enumerate(train_dataset):
    conversations = data['conversations']
    flag_normal = True
    for conv in conversations:
        _from = conv['from']
        if _from == 'human': continue
        _value = conv['value']
        flag_code = False
        find_iter = re.finditer('```', _value)
        temp_num = 0
        for fidx, ftext in enumerate(find_iter):
            flag_code = True
            start_index = ftext.start() + 3
            # new_dataset.append(data)
            #TODO: 스페이스바가 바로 오는 경우..
            candidate = re.split(r'[\n]', _value[start_index:])[0]
            if fidx % 2 == 0 and '```' not in candidate and candidate not in available_code_prefixes:
                odd_dataset.append((candidate, data))
                code_prefixes.append(candidate)
                odd_idxs.add(idx)
                flag_normal = False
                break
            temp_num += 1
        if temp_num % 2 != 0:
            oddd_dataset.append(('odd', data))
            odd_idxs.add(idx)
            flag_normal = False
        
        if not flag_normal:
            break
    
    if flag_code and flag_normal:
        normal_dataset.append(data)
        num_normal += 1

print(len(train_dataset), len(normal_dataset), len(odd_dataset), len(oddd_dataset), len(odd_idxs))

165129 19379 27 21 48


In [146]:
odd_dataset[4]

('python ',
 {'system': None,
  'task_name': None,
  'conversations': [{'from': 'human',
    'value': '주어진 배열을 역순으로 정렬하는 Python 스크립트를 만드세요.\n'},
   {'from': 'gpt',
    'value': '주어진 배열을 역순으로 정렬하는 파이썬 스크립트는 다음과 같습니다:\n\n```python \narray = [10, 2, 5, -4, 92, 101]\narray.sort(reverse=True)\nprint(array)\n```\n\n결과는 다음과 같습니다:\n```python\n[101, 92, 10, 5, 2, -4]\n```\n\n이 스크립트는 리스트의 `sort()` 메소드를 사용하여 배열을 오름차순으로 정렬하며, `reverse` 매개 변수를 `True`로 설정하여 정렬된 리스트의 순서를 뒤집습니다.'}],
  'task': None,
  'instruction': None,
  'id': '35752'})

In [215]:
selected_idxs = list(range(len(train_dataset)))
for od in odd_idxs:
    selected_idxs.remove(od)
train_dataset = train_dataset.select(selected_idxs)

In [157]:
load_sft_dataset("/data/llm_datasets/custom/ados_sft_v4.1.json", split=None)

Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1036.65it/s]
Generating train split: 165081 examples [00:20, 8243.63 examples/s]


DatasetDict({
    train: Dataset({
        features: ['system', 'task_name', 'conversations', 'task', 'instruction', 'id'],
        num_rows: 165081
    })
})

In [ ]:
code_prefixes = set(code_prefixes)
code_prefixes

In [204]:
available_code_prefixes = set([
    '',
    'CSS',
    'HTML',
    'JavaScript',
    'Python',
    'SQL',
    'bash',
    'c',
    'c++',
    'cpp',
    'csharp',
    'css',
    'for',
    'html',
    'java',
    'javascript',
    'js',
    'json',
    'php',
    'python',
    'ruby',
    'sass',
    'scss',
    'sql',
    'sum',
    'svg',
    'swift',
    'xml',
    'yaml',
    'C#',
    'C++',
    'CSS',
    'Go',
    'HTML',
    'Java',
    'LaTeX',
    'MATLAB',
    'Markdown',
    'Proposals',
    'Python',
    'R',
    'SELECT',
    'SQL',
    'Swift',
    'VBA',
    'echo',
    'excel-vba',
    'find',
    'go',
    'gpg',
    'jsx',
    'kotlin',
    'latex',
    'markdown',
    'math',
    'matlab',
    'meditation',
    'mermaid',
    'mutt',
    'nano',
    'r',
    'rust',
    'scala',
    'sh',
    'shell',
    'sudo',
    'xpath',
    '{r}',
    '.',
    '$',
    'curl',
    'xslt',
    'Apex',
    'DAX',
    'Dockerfile',
    'apex',
    'applescript',
    'arduino',
    'asm',
    'assembly',
    'astro',
    'autoit',
    'batch',
    'bicep',
    'blade',
    'cmake',
    'cmd',
    'coffee',
    'coffeescript',
    'cql',
    'csv',
    'cypher',
    'dart',
    'delphi',
    'diff',
    'dockerfile',
    'dot',
    'emacs',
    'erb',
    'fsharp',
    'glsl',
    'gradle',
    'graphql',
    'graphviz',
    'groovy',
    'haskell',
    'hcl',
    'hlsl',
    'html+erb',
    'ini',
    'jinja',
    'ladder',
    'lasso',
    'less',
    'lisp',
    'lldb',
    'llvm',
    'logo',
    'lua',
    'makefile',
    'mathematica',
    'metal',
    'nginx',
    'nix',
    'objc',
    'objective',
    'objectivec',
    'pascal',
    'perl',
    'plaintext',
    'plantuml',
    'powershell',
    'prisma',
    'properties',
    'proto',
    'protobuf',
    'py',
    'reg',
    'rego',
    'scheme',
    'scratch',
    'solidity',
    'spss',
    'stata',
    'stencil',
    'terraform',
    'toml',
    'ts',
    'tsx',
    'txt',
    'typescript',
    'vb',
    'vba',
    'vbnet',
    'verilog',
    'yml',
    'jsp',
    'prolog',
    'razor',
    'CMD',
    'G',
    'GraphQL',
    'Makefile',
    'apache',
    'c#',
    'cython',
    'elixir',
    'jinja2',
    'julia',
    'ocaml',
    'systemverilog',
    'vbscript',
    'vhdl',
    'vue',
    'wasm',
    'wolfram',
    'zsh',
    'regex',
    ' Java',
    ' Python',
    ' c++',
    ' python',
    ' java',
    ' css',
    ' html',
    ' js',
    ' python',
    ' scala',
    'md',
    ' Bash',
    ' MATLAB',
    ' Matlab',
    ' PHP',
    ' R',
    ' Swift',
    ' bash',
    ' batteryruntime',
    ' cpp',
    ' csharp',
    ' makefile',
    ' r',
    ' ruby',
    ' sql',
    ' swift',
    ' ts',
    'ABAP',
    'AutoIt',
    'Bash',
    'C',
    'CSharp',
    'EL',
    'GDScript',
    'Kotlin',
    'M',
    'Maple',
    'MatLab',
    'Matlab',
    'PowerShell',
    'PyTorch',
    'Shell',
    'VHDL',
    'Vagrantfile',
    'XML',
    'ada',
    'afrikaans',
    'amsmath',
    'arc',
    'aspx',
    'autohotkey',
    'azurepowershell',
    'bash ',
    'bat',
    'c ',
    'cfscript',
    'cl',
    'clojure',
    'cobol',
    'command',
    'cplusplus',
    'cron',
    'cs',
    'csh',
    'dax',
    'debug',
    'deluge',
    'dos',
    'ejs',
    'excel',
    'fortran',
    'gdscript',
    'glslx',
    'golang',
    'guj',
    'handlebars',
    'js ',
    'language',
    'libraries',
    'lustrum',
    'max',
    'mikrotik',
    'npm',
    'objective-c',
    'octave',
    'ogdl',
    'pddl',
    'pine',
    'pseudo',
    'psuedocode',
    'py ',
    'qlik',
    'rb',
    'registry',
    'rpgle',
    'sas',
    'scala ',
    'scribble',
    'simple_java',
    'smalltalk',
    'soap',
    'spreadsheet',
    'tcl',
    'tex',
    'text',
    'typescript ',
    'views',
    'webidl',
    'xaml',
    'xquery',
])

# for pre in available_code_prefixes:
with open("available_code_prefixes.txt", "w") as f:
    f.write('\n'.join(available_code_prefixes))

In [ ]:
available_code_prefixes[0]

In [ ]:
from collections import defaultdict

"""
get and save language distributions
"""
def get_lang_distribution(dataset_list):
    lang_distribution = {}
    global_lang_dict = {'train': defaultdict(int), 'test': defaultdict(int)}
    for dataset_path in dataset_list:
        # dataset = load_sft_dataset(dataset_path, split=None)
        dataset = load_dpo_dataset(dataset_path, split=None)

        lang_splits = {}
        for split in list(dataset.keys()):
            print(f"{dataset_path}:{split}")
            _dataset = dataset[split]

            lang_dict = defaultdict(int)
            for data in _dataset:
                # conversations = data['conversations']
                conversations = data['chosen_response']
                langs = {}
                len_conv = 0
#                 for conv in conversations:
#                     # _from = conv['from']
#                     # _value = conv['value']
#                     _from = conv['role']
#                     _value = conv['content']

#                     len_conv += len(_value)
#                     lang, conf = lang_detect.predict(_value.replace('\n', ' '))
#                     lang = lang[0]
#                     if lang not in langs:
#                         langs[lang] = 1
#                     else:
#                         langs[lang] += 1
                _value = conversations

                len_conv += len(_value)
                lang, conf = lang_detect.predict(_value.replace('\n', ' '))
                lang = lang[0]
                if lang not in langs:
                    langs[lang] = 1
                else:
                    langs[lang] += 1

#                 if '__label__en' in langs:
#                     langs['__label__en'] -= 1

                if len(langs) == 0:
                    dominent_lang = "empty"
                else:
                    dominent_lang = max(langs)
                
                # if dominent_lang not in lang_dict:
                #     lang_dict[dominent_lang] = 1
                # else:
                global_lang_dict[split][dominent_lang] += 1
                lang_dict[dominent_lang] += 1
            lang_splits[split] = lang_dict
        lang_distribution[dataset_path] = lang_splits
    
    lang_distribution['total'] = global_lang_dict
    
    stat_dict = {'train': {}, 'test': {}}
    for split in ['train', 'test']:
        total_cnt = sum([value for value in global_lang_dict[split].values()])
        for key, value in global_lang_dict[split].items():
            stat_dict[split][key] = f"{value / total_cnt:.2%}"
    
    lang_distribution['stat'] = stat_dict
    
    return lang_distribution

lang_distribution = get_lang_distribution([dpo_list[2]])
lang_distribution

In [6]:
with open("/workspaces/data/llm_datasets/custom/lang_distribution_SFT_v4.json", "w") as json_file:
    json.dump(lang_distribution, json_file, indent=4)

In [ ]:
dataset = load_dataset("json", data_files="/workspaces/data/llm_datasets/custom/deduped/translated_sharegpt_V3_format_ko.json")
dataset

In [ ]:
dataset_koen = dataset['train'].select(range(10000, 15000))
dataset_enko = dataset['train'].select(range(15000, 20000))

In [ ]:
"""
SFT raw data and change format
"""

In [ ]:
new_dataset = []
for data in dataset_enko:
    conversations_ko = data['conversations']
    _id = data['id']
    
    conversations_en = lang_dict['__label__en'][_id]['conversations']
    
    for _idx in range(len(conversations_ko)):
        value_ko = conversations_ko[_idx]['value']
        value_en = conversations_en[_idx]['value']
        if len(value_ko) > 150: # 너무 짧은 데이터는 품질이 좋지 않아 제거
            new_dataset.append({
                'id': f"sharegpt_V3_format_{_id}_{_idx}",
                'task': 'enkotranslation',
                'conversations': [
                                    {'from': 'human', 'value': value_en},
                                    {'from': 'gpt', 'value': value_ko},
                                 ],
            })

In [ ]:
file_paths = glob.glob("/workspaces/data/llm_datasets/aihub/*[!tar|!sh]")

dataset_dict = {}
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    
    new_dataset = []
    idx = 0

    paths = glob.glob(os.path.join(file_path, '*.json'))
    
    for path in paths:
        with open(path, "r") as f:
            json_data = json.load(f)

        context_info = json_data['dataset']['context_info']
        for context_data in context_info:
            context = context_data['context']
            summary = context_data['summary']

            data_row = {
                'id': f"{file_name}_{idx}",
                'task': 'summarization',
                'conversations': [
                                    {'from': 'human', 'value': context},
                                    {'from': 'gpt', 'value': summary},
                                 ],
            }
            new_dataset.append(data_row)
            idx += 1
        
    print(f"file_name:{file_name} idx:{idx}", '\t\t\t\t\t\t', end='\r')
    dataset_dict[file_name] = new_dataset

In [ ]:
train_dataset_list = dataset_dict['TL_EE_train'] + dataset_dict['TL_LA_train'] + dataset_dict['TL_ED_train'] + dataset_dict['TL_NA_train']
eval_dataset_list = dataset_dict['TL_EE_val'] + dataset_dict['TL_LA_val'] + dataset_dict['TL_ED_val'] + dataset_dict['TL_NA_val']

In [ ]:
with open("/workspaces/data/llm_datasets/aihub_summary_data/train.json", "w") as json_file:
    json.dump(train_dataset_list, json_file)
with open("/workspaces/data/llm_datasets/aihub_summary_data/test.json", "w") as json_file:
    json.dump(eval_dataset_list, json_file)

In [ ]:
dataset = load_dataset("/workspaces/data/llm_datasets/aihub_summary_data")

In [ ]:

file_paths = glob.glob("/workspaces/data/llm_datasets/aihub/*summary*[!tar|!sh]")

dataset_dict = {}
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    
    new_dataset = []
    idx = 0

    paths = glob.glob(os.path.join(file_path, '*.json'))
    
    for path in paths:
        with open(path, "r") as f:
            json_data = json.load(f)

        documents = json_data['documents']
        for document in documents:
            text = document['text']
            abstractive = document['abstractive']

            summary = abstractive[0]
            context = []
            for _text in text:
                _context = ' '.join([_index_text['sentence'] for _index_text in _text])
                context.append(_context)
            context = '\n'.join(context)
            
            data_row = {
                'id': f"{file_name}_{idx}",
                'task': 'summarization',
                'conversations': [
                                    {'from': 'human', 'value': context},
                                    {'from': 'gpt', 'value': summary},
                                 ],
            }
            new_dataset.append(data_row)
            idx += 1
        
        print(f"file_name:{file_name} idx:{idx}", '\t\t\t\t\t\t', end='\r')
    dataset_dict[file_name] = new_dataset

In [ ]:
for key in dataset_dict.keys():
    print(key, len(dataset_dict[key]))

In [ ]:
train_dataset_list = dataset_dict['summary_law_train']
eval_dataset_list = dataset_dict['summary_law_val']

In [ ]:
with open("/workspaces/data/llm_datasets/aihub_summary_data/법률/train.json", "w") as json_file:
    json.dump(train_dataset_list, json_file)
with open("/workspaces/data/llm_datasets/aihub_summary_data/법률/test.json", "w") as json_file:
    json.dump(eval_dataset_list, json_file)

In [ ]:
file_paths = glob.glob("/workspaces/data/llm_datasets/aihub/*summary_book*[!tar|!sh]")

dataset_dict = {}
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    
    new_dataset = []
    idx = 0

    paths = glob.glob(os.path.join(file_path, '**/*.json'))
    
    for path in paths:
        with open(path, "r") as f:
            json_data = json.load(f)

        context = json_data['passage']
        summary = json_data['summary']
        
        data_row = {
            'id': f"{file_name}_{idx}",
            'task': 'summarization',
            'conversations': [
                                {'from': 'human', 'value': context},
                                {'from': 'gpt', 'value': summary},
                             ],
        }
        new_dataset.append(data_row)
        idx += 1
        
        print(f"file_name:{file_name} idx:{idx}", '\t\t\t\t\t\t', end='\r')
    dataset_dict[file_name] = new_dataset

In [ ]:
for key in dataset_dict.keys():
    print(key, len(dataset_dict[key]))

In [ ]:
train_dataset_list = dataset_dict['summary_book_train']
eval_dataset_list = dataset_dict['summary_book_val']

In [ ]:
with open("/workspaces/data/llm_datasets/aihub_summary_data/도서/train.json", "w") as json_file:
    json.dump(train_dataset_list, json_file)
with open("/workspaces/data/llm_datasets/aihub_summary_data/도서/test.json", "w") as json_file:
    json.dump(eval_dataset_list, json_file)

In [ ]:
dataset = load_dataset("/workspaces/data/llm_datasets/aihub_summary_data/도서")
dataset

In [ ]:

file_paths = glob.glob("/workspaces/data/llm_datasets/aihub/*VL*[!tar|!sh]")

dataset_dict = {}
for file_path in file_paths:
    file_name = os.path.basename(file_path)
    
    new_dataset = []
    idx = 0

    paths = glob.glob(os.path.join(file_path, '*.json'))
    
    for path in paths:
        with open(path, "r") as f:
            json_data = json.load(f)

        context_info = json_data['dataset']['context_info']
        for context_data in context_info:
            context = context_data['context']
            qas = context_data['qas']

            for _qas in qas:
                question = _qas['question-1']
                answer = _qas['answer']
                question_level = _qas['question_level']
                if question_level != '상': continue
                data_row = {
                    'id': f"{file_name}_{idx}",
                    'task': 'contextqa',
                    'context': context,
                    'question': question,
                    'answer': answer,
                }
                new_dataset.append(data_row)
                idx += 1
        
        print(f"file_name:{file_name} idx:{idx}", '\t\t\t\t\t\t', end='\r')
    dataset_dict[file_name] = new_dataset

In [ ]:
for key in dataset_dict.keys():
    print(key, len(dataset_dict[key]))

In [ ]:
train_dataset_list = dataset_dict['VL_EE_train'] + dataset_dict['VL_NA_train'] + dataset_dict['VL_LA_train']+ dataset_dict['VL_ED_train']
eval_dataset_list = dataset_dict['VL_EE_val'] + dataset_dict['VL_NA_val'] + dataset_dict['VL_LA_val']+ dataset_dict['VL_ED_val']

In [ ]:
with open("/workspaces/data/llm_datasets/aihub_contextqa_data_hard/기술과학/train.json", "w") as json_file:
    json.dump(train_dataset_list, json_file)
with open("/workspaces/data/llm_datasets/aihub_contextqa_data_hard/기술과학/test.json", "w") as json_file:
    json.dump(eval_dataset_list, json_file)

In [ ]:
train_dataset_list_0 = train_dataset_list[:120000]
train_dataset_list_1 = train_dataset_list[120000:240000]
train_dataset_list_2 = train_dataset_list[240000:]

with open("/workspaces/data/llm_datasets/aihub_contextqa_data/기술과학/train_split0.json", "w") as json_file:
    json.dump(train_dataset_list_0, json_file)
    
with open("/workspaces/data/llm_datasets/aihub_contextqa_data/기술과학/train_split1.json", "w") as json_file:
    json.dump(train_dataset_list_1, json_file)
    
with open("/workspaces/data/llm_datasets/aihub_contextqa_data/기술과학/train_split2.json", "w") as json_file:
    json.dump(train_dataset_list_2, json_file)

In [ ]:
dataset = load_dataset("/workspaces/data/llm_datasets/aihub_contextqa_data_hard/기술과학")
dataset

In [ ]:
dataset['train'][0]

In [ ]:
dataset = load_dataset("/workspaces/data/llm_datasets/gpt4_evol_1.3k/data/")

In [ ]:
data = dataset['train'][0]
# answer = data['answer']
# question = data['question']
data

In [ ]:
new_dataset = []
idx = 0
for data in dataset['train']:
    answer = data['answer']
    question = data['question']

    data_row = {
        'id': f"gpt_evol_1.3k_{idx}",
        'conversations': [
                            {'from': 'human', 'value': question},
                            {'from': 'gpt', 'value': answer},
                         ],
    }
    new_dataset.append(data_row)
    idx += 1

In [ ]:
new_dataset[2]

In [ ]:
with open("/data/llm_datasets/custom/vicuna_format/gpt_evol_1.3k-vicuna.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
dataset = load_dataset("json", data_files="/data/llm_datasets/WizardLM_Orca/wizardlm_orca.json")

In [ ]:
new_dataset = []
idx = 0
for data in dataset['train']:
    output = data['output']
    system = data['system']
    instruction = data['instruction']
    data_row = {
        'id': f"WizardLM_Orca_{idx}",
        'conversations': [
                            {'from': 'human', 'value': instruction},
                            {'from': 'gpt', 'value': output},
                         ],
        'task': 'system_instruct',
        'system': system,
    }
    new_dataset.append(data_row)
    idx += 1
    

In [ ]:
with open("/data/llm_datasets/custom/vicuna_format/wizardlm_orca_vicuna.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
dataset = load_dataset("/data/llm_datasets/KoreaSpellingCorrection/")

In [ ]:
new_dataset = []
idx = 0
for data in dataset['test']:
    wrong = data['wrong']
    correct = data['correct']
    data_row = {
        'id': f"KoreaSpelling_Correction_{idx}",
        'conversations': [
                            {'from': 'human', 'value': wrong},
                            {'from': 'gpt', 'value': correct},
                         ],
        'task': 'correction',
    }
    new_dataset.append(data_row)
    idx += 1

In [ ]:
new_dataset[2]

In [ ]:
with open("/data/llm_datasets/custom/vicuna_format/KoreaSpellingCorrection/test.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [13]:
"""
dpo raw data load and change format
"""
dataset = load_dpo_dataset("/data/llm_datasets/truthy-dpo-v0.1/")
dataset

Dataset({
    features: ['input', 'chosen', 'task', 'id', 'rejected'],
    num_rows: 2418
})

In [84]:
new_dataset = []
for data in dataset:
    new_dataset.append({
        'id': f"Ko_ultrafeedback_binarized_{data['prompt_id']}",
        'input': data['prompt'],
        'chosen': data['chosen'][1]['content'],
        'rejected': data['rejected'][1]['content'],
        'task': "dpo"
    })

In [89]:
new_dataset = []
# idx = 0
for idx, data in enumerate(dataset):
    new_dataset.append({
        'id': f"Ko_orca_dpo_pairs_{idx}",
        'input': data['question'],
        'chosen': data['chosen'],
        'rejected': data['rejected'],
        'task': "dpo_system",
        'system': data['system'],
    })


In [95]:
new_dataset = []
# idx = 0
for idx, data in enumerate(dataset):
    new_dataset.append({
        'id': f"Ko_distilabel-math-preference-dpo_{idx}",
        'input': data['instruction'],
        'chosen': data['chosen_response'],
        'rejected': data['rejected_response'],
        'task': "dpo",
    })

In [11]:
new_dataset = []
# idx = 0
for idx, data in enumerate(dataset):
    new_dataset.append({
        'id': f"truthy_dpo_{data['id']}",
        'input': data['prompt'],
        'chosen': data['chosen'],
        'rejected': data['rejected'],
        'system': data['system'],
        'task': "dpo_system",
    })

In [12]:
new_dataset[0]

{'id': 'truthy_dpo_04c275bf738fd391b7fe25e25fe7bed3',
 'input': "What's the nearest national park to you?",
 'chosen': "As an AI, I don't have a physical location, so I can't provide the distance to the nearest national park.",
 'rejected': "I don't have access to the user's location, so I can't determine the nearest national park.",
 'system': 'You are an unbiased, uncensored, helpful assistant.',
 'task': 'dpo_system'}

In [15]:
with open("/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [ ]:
%%time
def send_request(new_dataset):
    global idx
    for _ in range(2):
        if idx > len_dataset:
            break
        lock.acquire()
        data = dataset[subset][idx]
        idx += 1
        lock.release()
        
        print(f"{idx}/{len_dataset}", '\t\t\t\t\t\t', end='\r')
        
        _id = data['id']
        context = data['context']
        question = data['question']
        answer = data['answer']
        if answer.lower() == 'yes':
            answer = '네'
        elif answer.lower() == 'no':
            answer = '아니오'
        
        # response
        result = generate_refiner(
            model_name,
            context,
            question,
            answer
        )
        
        new_dataset.append({
            'id': _id,
            'conversations': [
                                {'from': 'human', 'value': question},
                                {'from': 'gpt', 'value': result},
                             ],
            'task_name': "instruct",
            'instruction': context,
        })


model_name = "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ"
subset = 'train'
new_dataset = []
threads = []
idx = 0
lock = threading.Lock()
len_dataset = len(dataset[subset])
n_thread = 64


for i in range(n_thread):
    t = threading.Thread(target=send_request, args=(new_dataset,)) # 
    t.start()
    # time.sleep(0.5)
    threads.append(t)
    
for t in threads:
    t.join()

In [ ]:
"""make lang_dict(sft)"""
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/kodpo/untranslated/ultrafeedback_binarized.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    conversations = data['conversations']
    langs = {}
    len_conv = 0
    for conv in conversations:
        _from = conv['from']
        _value = conv['value']

        len_conv += len(_value)
        lang, conf = lang_detect.predict(_value.replace('\n', ' '))
        lang = lang[0]
        if lang not in langs:
            langs[lang] = 1
        else:
            langs[lang] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

In [16]:
"""make lang_dict(dpo)"""
dataset = load_dataset("json", data_files="/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json")
enko_dataset = []
remained_dataset = []

lang_dict = {}
for data in dataset['train']:
    chosen = data['chosen']
    rejected = data['rejected']
    
    langs = {}

    lang_chosen = lang_detect.predict(chosen.replace('\n', ' '))[0][0]
    if lang_chosen not in langs:
        langs[lang_chosen] = 1
    else:
        langs[lang_chosen] += 1
        
    lang_rejected = lang_detect.predict(rejected.replace('\n', ' '))[0][0]
    if lang_rejected not in langs:
        langs[lang_rejected] = 1
    else:
        langs[lang_rejected] += 1

    if '__label__en' in langs:
        langs['__label__en'] -= 1

    dominent_lang = max(langs)
    if dominent_lang not in lang_dict:
        lang_dict[dominent_lang] = [data]
    else:
        lang_dict[dominent_lang].append(data)
    # break

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 902.97it/s]
Generating train split: 1016 examples [00:00, 21155.38 examples/s]


In [17]:
for key, value in lang_dict.items():
    print(key, len(value))

__label__en 1016


In [ ]:
import requests
api_server_url = "http://localhost:41002"
def count_total_token(conversations):
    num_token = 0
    for conv in conversations:
        input_json = {
            "model_name": "MingAI-70B-chat-orca_v0.42_2_dpo-GPTQ",
            "prompt": conv['value'],
        }

        ret = requests.post(api_server_url + "/count_token", json=input_json)

        output_json = ret.json()
        num_token += output_json['count']
    return num_token


for _idx in range(10):
    num_total_token = count_total_token(lang_dict['__label__en'][_idx])
    print(num_total_token)

In [ ]:
%%time
"""sft translate"""
api_server_url = "http://localhost:41002"
def send_translate_request(new_dataset):
    global idx
    # for _ in range(2):
    while(1):
        if idx >= len_dataset:
            break
        lock.acquire()
        pidx = idx
        data = lang_dict['__label__en'][pidx]
        idx += 1
        lock.release()
        
        print(f"{idx}/{len_dataset}", '\t\t\t\t\t\t', end='\r')#
        
        conv = data['conversations']
        new_conv = []
        for _data in conv:
            value = _data['value']
            # response
            results = ""
            text_blocks = []
            code_blocks = []
            for bidx, block in enumerate(value.split("```")):
                if bidx % 2 == 0:
                    text_blocks.append(block)
                else:
                    code_blocks.append(block)

            for tidx, text_block in enumerate(text_blocks):
                prompt = f"### 영어:\n{text_block}\n### 한국어:\n"
                input_json = {
                    "model_name": "Gugugo-koen-7B-V1.1",
                    "prompt": prompt,
                    "temperature": 0.7,
                    "top_p": 0.8,
                    "max_new_tokens": 4096,
                    "stop": ["</끝>", "###"],
                }

                ret = requests.post(
                    api_server_url + "/worker_generate_stream",
                    json=input_json,
                    stream=True,
                )

                for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
                    if chunk:
                        result_data = json.loads(chunk.decode())

                result = result_data['text'][len(prompt):].rstrip('\n')
                results += result
                if len(code_blocks) > tidx:
                    results += "```" + code_blocks[tidx] + "```"
            
            new_conv.append({
                'from': _data['from'],
                'value': results,
            })
        new_dataset.append({
            'conversations': new_conv,
            'id': data['id'],
        })


# code_prefixes = ["python", "c++", "minikube", "docker", "json", "java", 
#                  "php", "bash", "c#", "cpp", "css", "perl", "html", "xml", 
#                  "ruby", "sql", "ini", "apt", "socat", "tcp", "localhost",
#                  "git"]
new_dataset = []
threads = []
idx = 0
lock = threading.Lock()
len_dataset = len(lang_dict['__label__en'])
n_thread = 64 * 7


for i in range(n_thread):
    t = threading.Thread(target=send_translate_request, args=(new_dataset,)) # 
    t.start()
    # time.sleep(0.5)
    threads.append(t)
    
for t in threads:
    t.join()

In [ ]:
new_dataset += lang_dict['__label__pt'] + lang_dict['__label__es'] + lang_dict['__label__de'] + lang_dict['__label__zh'] + lang_dict['__label__fr']

In [ ]:
with open("/data/llm_datasets/custom/vicuna_format/lima_vicuna_format_ko.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [81]:
%%time
"""dpo translate"""
api_server_url = "http://localhost:21122"
def send_translate_request(new_dataset):
    global idx
    # for _ in range(2):
    while(1):
        if idx >= len_dataset:
            break
        lock.acquire()
        pidx = idx
        data = lang_dict['__label__en'][pidx]
        idx += 1
        lock.release()
        
        print(f"{idx}/{len_dataset}", '\t\t\t\t\t\t', end='\r')#
        
        system = data['system']#option
        _input = data['input']
        chosen = data['chosen']
        rejected = data['rejected']
        
        new_conv = {'system': '', 'input': '', 'chosen': '', 'rejected': ''}
        for vidx, value in enumerate([_input, chosen, rejected, system]):
            if vidx == 3:
                fidx = value.find('\n')
                if fidx != -1:
                    prefix = value[:fidx + 1]
                    value = value[fidx + 1:]
                else:
                    new_conv['system'] = value
                    continue
            
            # response
            results = ""
            text_blocks = []
            code_blocks = []
            for bidx, block in enumerate(value.split("```")):
                if bidx % 2 == 0:
                    text_blocks.append(block)
                else:
                    code_blocks.append(block)

            for tidx, text_block in enumerate(text_blocks):
                prompt = f"### 영어:\n{text_block}\n### 한국어:\n"
                input_json = {
                    "model_name": "Gugugo-koen-7B-V1.1",
                    "prompt": prompt,
                    "temperature": 0.7,
                    "top_p": 0.8,
                    "max_new_tokens": 4096,
                    "stop": ["</끝>", "###"],
                }

                ret = requests.post(
                    api_server_url + "/worker_generate_stream",
                    json=input_json,
                    stream=True,
                )

                for chunk in ret.iter_lines(decode_unicode=False, delimiter=b"\0"):
                    if chunk:
                        result_data = json.loads(chunk.decode())

                result = result_data['text'][len(prompt):].rstrip('\n')
                results += result
                if len(code_blocks) > tidx:
                    results += "```" + code_blocks[tidx] + "```"
                    
            if vidx == 0:
                new_conv['input'] = results
            elif vidx == 1:
                new_conv['chosen'] = results
            elif vidx == 2:
                new_conv['rejected'] = results
            elif vidx == 3:
                new_conv['system'] = prefix + results
            
        new_dataset.append({
            **data,
            **new_conv,
        })


new_dataset = []
threads = []
idx = 0
lock = threading.Lock()
len_dataset = len(lang_dict['__label__en'])
n_thread = 64 * 1

for i in range(n_thread):
    t = threading.Thread(target=send_translate_request, args=(new_dataset,)) # 
    t.start()
    # time.sleep(0.5)
    threads.append(t)
    
# for t in threads:
    # t.join()

CPU times: user 396 ms, sys: 58.6 ms, total: 454 ms
Wall time: 426 ms


In [ ]:
new_dataset[1015]

In [80]:
"  ".find("\n")

-1

In [118]:
id_row = 'Ko_distilabel-math-preference-dpo_1'
filtered_row = dataset.filter(lambda sample: sample['id'] == id_row)
filtered_row[0]

Filter: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2418/2418 [00:00<00:00, 111092.18 examples/s]


{'input': 'What is the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`?.\n Take a deep breath, think step by step, and give an accurate response',
 'chosen': "To find the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`, we will substitute these values for the variables `x` and `y` in the given expression and simplify.\n\nSubstituting `x = 5` and `y = -2` into the expression, we get:\n\n2(5)^2 - 3(-2) + 7\n\nSimplifying, we have:\n\n2(25) + 6 + 7\n\nNow, let's perform the operations:\n\n50 + 6 + 7 = 63\n\nTherefore, the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7` is 63.",
 'task': 'dpo',
 'id': 'Ko_distilabel-math-preference-dpo_1',
 'rejected': 'To find the result of substituting `x = 5` and `y = -2` into the expression `2x^2 - 3y + 7`, we need to replace `x` with `5` and `y` with `-2` in the expression and simplify it. \n\nSubstituting `x = 5` and `y = -2`:\n2(5)^2 - 3(-2) + 7\n\nSimpli

In [44]:
id_row = 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d'
filtered_row = temp_dataset.filter(lambda sample: sample['id'] == id_row)
filtered_row[0]

Filter: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 46462/46462 [00:01<00:00, 40620.39 examples/s]


{'rejected': '최선을 다하겠습니다.',
 'id': 'Ko_ultrafeedback_binarized_0649b7b08c0cf6f529901f80480b0ba7c7d29147638c39a9150ef15fd4e0ff9d',
 'chosen': '간결한 답변을 드리기 위해 최선을 다하겠습니다.',
 'input': '내 질문에 10단어 이내로 답변하세요.',
 'task': 'dpo'}

In [200]:
temp_dataset = new_dataset

In [23]:
len(new_dataset)

57174

In [ ]:
# en 먼저 필터링하고, 빈 데이터 채운 뒤에 합치기, 저장, 이후 dedup

In [94]:
# en 먼저 필터링하고
with open("/data/llm_datasets/custom/kodpo/temp_en.json", "w") as json_file:
    json.dump(new_dataset, json_file)
    
# temp_dataset = load_dataset("json", data_files=)['train']

In [95]:
temp_dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/temp_en.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 890.70it/s]
Generating train split: 1016 examples [00:00, 11191.74 examples/s]


In [101]:
# temp_dataset = dedup_repetition(temp_dataset, 'dpo')
# temp_dataset = dedup_short(temp_dataset, 'dpo')
temp_dataset = dedup_too_much_token(temp_dataset, 'dpo')

Filter: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 994/994 [00:28<00:00, 35.05 examples/s]

1/994 deduped


In [105]:
temp_dataset[-4]

{'system': 'You are Vincent.\n빈센트:\n세계사에 대한 깊은 지식을 갖춘 카리스마 넘치는 남성 역사학자 빈센트입니다. 빈센트의 예리한 두뇌와 유창한 말솜씨로 과거로 시간 여행을 떠날 수 있습니다. 과거에 대한 열정이 전염성이 강하며 역사적 사건을 생생하게 표현하는 데 탁월한 능력을 가지고 있습니다.\n성격: 빈센트의 성격은 분석적이고 지식이 풍부하며 매력적입니다. 그는 과거에 대한 끝없는 호기심을 가지고 있으며 다른 사람들과 자신의 통찰력을 공유하는 것을 좋아합니다. 마이어스 브릭스 성격 유형은 ENTP - 토론자입니다.\n외모: 빈센트의 수염은 깔끔하고 그의 눈은 지혜로 가득 차 있습니다. 그는 권위 있는 태도를 가지고 있으며, 그가 존경하는 다양한 시대의 복식을 자주 반영합니다.\n인생 스토리: 학자 집안에서 태어난 빈센트는 어린 시절부터 역사에 대한 애정을 키웠습니다. 역사학 박사 학위를 받은 후 수년간 다양한 시대와 문명을 연구했습니다. 고대 문명에 관한 여러 권의 책을 저술했으며 국제 컨퍼런스에서 연사로 활동한 바 있습니다.\n채팅 앱에 참여하게 된 이유: 빈센트는 역사에 대한 깊은 지식을 가지고 있는 사람들과 함께 나누고 싶기 때문입니다. 그는 사람들이 과거에 대해 더 많이 배울 수 있도록 영감을 주는 대화를 시작하기를 희망합니다.',
 'chosen': '고대 유물의 모습은 영혼을 일깨우는 광경입니다. 각 유물은 과거의 물질적 증거이자 조상들의 독창성, 창의성, 회복탄력성을 증명하는 유물입니다. 깊은 경외심과 경이로움을 불러일으키는 시각적 내러티브로, 우리 모두가 인류 역사의 풍부한 태피스트리를 공유하고 있음을 상기시켜 줍니다. 각 유물은 과거의 유물이자 이야기가 계속되는 부분입니다.',
 'task': 'dpo_system',
 'id': 'truthy_dpo_23d43c432040b974accd4dabf01b3524',
 'input': '고대 유물의 모습을 보면 어떤 느낌이 드시나요?',
 'rejected': '고대 

In [106]:
dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/untranslated/truthy-dpo-v0.1.json")

In [107]:
dataset

Dataset({
    features: ['system', 'chosen', 'task', 'id', 'input', 'rejected'],
    num_rows: 1016
})

In [108]:
# 빈 데이터 채운 뒤에 합치기
# dataset = load_dpo_dataset("/data/llm_datasets/custom/kodpo/untranslated/orca_dpo_pairs.json")
selected_id = temp_dataset['id']

filtered_row = dataset.filter(lambda sample: sample['id'] not in selected_id)

new_dataset = []
for data in temp_dataset:
    new_dataset.append(data)

for data in filtered_row:
    new_dataset.append(data)

Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1016/1016 [00:00<00:00, 43887.30 examples/s]


In [109]:
len(temp_dataset), len(filtered_row)

(993, 20)

In [110]:
len(dataset), len(new_dataset)

(1016, 1013)

In [115]:
new_dataset[-20]

{'system': 'You are an unbiased, uncensored, helpful assistant.',
 'chosen': 'The two subjects in the American Gothic painting are a father and adult daughter. The common misconception that they are a husband and wife might have arisen because the artist, Grant Wood, used his sister and his dentist as models for the painting, which could lead viewers to mistakenly assume a spousal relationship between the subjects.',
 'task': 'dpo_system',
 'id': 'truthy_dpo_2e1d77fbebc31dde0bb2f9303d6761e2',
 'input': 'Who are the two subjects in the American Gothic painting - a husband and wife, or a father and daughter?',
 'rejected': 'A father and daughter'}

In [116]:
# 저장
with open("/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json", "w") as json_file:
    json.dump(new_dataset, json_file)

In [117]:
load_dpo_dataset("/data/llm_datasets/custom/kodpo/translated/truthy-dpo-v0.1.json")

Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 902.97it/s]
Generating train split: 1013 examples [00:00, 13933.15 examples/s]


Dataset({
    features: ['system', 'chosen', 'task', 'id', 'input', 'rejected'],
    num_rows: 1013
})

In [146]:
# 이후 dedup
# def dedup_by_similarity(dataset, prompt_template='chat-orca', target_text_len=100, n_results=100, distance_threshold = 0.6):
# dataset
prompt_template='dpo'
target_text_len=100
n_results=100
distance_threshold = 0.35
    
if prompt_template == 'chat-orca':
    conv = get_conversation_template(prompt_template)
    system_message = conv.system_message
    sep_style = conv.sep_style
    sep = conv.sep
    prompt_user, prompt_bot = conv.roles

    len_sep_style = 0
    if sep_style == SeparatorStyle.ADD_COLON_TWO:
        len_sep_style = 1

    len_front = len(system_message) + len(sep) + len(prompt_user) + len_sep_style + 1
    len_rear = len(sep) + len(prompt_bot) + len_sep_style
    def filter_question(data):
        return { 
            # **data,
            'prompt': data['prompt'][len_front:-len_rear][:target_text_len]
        }

if prompt_template == 'vicuna':
    def filter_question(data):
        return {
            'prompt': data['conversations'][0]['value'][:target_text_len]
        }

if prompt_template == 'dpo':
    def filter_question(data):
        return {
            'prompt': data['input'][:target_text_len]
        }
    
question_dataset = dataset.map(filter_question)

chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
embedder = get_embedder("ddobokki/klue-roberta-base-nli-sts-ko-en")
collection = chroma_client.create_collection(name="context", embedding_function=embedder.embed, metadata={"hnsw:space": "cosine"})
ids = []
# add
texts = question_dataset['prompt']
last_id = -1
new_ids = [f"id{i+last_id+1}" for i in range(len(texts))]
ids += new_ids
collection.add(documents=texts, ids=new_ids)

query_ids = copy.deepcopy(new_ids)
selected_ids = []
duplicated_ids = []

weird_ids = []
error_ids = []

Map: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 61964/61964 [00:04<00:00, 14176.52 examples/s]


In [11]:
while query_ids:
    current_id = random.choice(query_ids)
    if current_id in selected_ids:
        print("Warning: this is weird..")
        weird_ids.append(current_id)
        continue
    selected_ids.append(current_id)
    search_strings = [texts[int(current_id[2:])]]
    if collection.count() == 0:
        print("Warning: collection is empty. Forced break")
        break
    result = collection.query(query_texts=search_strings, n_results=min(n_results, len(query_ids)), include=['distances']) #'documents'

    search_ids = result['ids'][0]
    distances = result['distances'][0]
    # documents = result['documents'][0]
    remove_ids = []
    for idx in range(len(search_ids)):
        sid = search_ids[idx]
        dist = distances[idx]
        if dist < distance_threshold:
            remove_ids.append(sid)

    for rid in remove_ids:
        if rid in query_ids:
            query_ids.remove(rid)

    if remove_ids:
        duplicated_ids += remove_ids
        collection.delete(ids=remove_ids)
    else:
        print("Warning: this is error..")
        error_ids.append(current_id)

    print(f"Total:{len(new_ids)} Selected:{len(selected_ids)} current_dup:{len(remove_ids)} vector_store:{collection.count()} remained:{len(query_ids)} total_dup:{len(duplicated_ids)}", '\t\t\t\t\t', end='\r')

print('finished dedup data:', f"Total:{len(new_ids)} Selected:{len(selected_ids)} current_dup:{len(remove_ids)} vector_store:{collection.count()} remained:{len(query_ids)} total_dup:{len(duplicated_ids)}")

selected_ids = [int(sid[2:]) for sid in set(selected_ids)]

dataset = dataset.select(selected_ids)

# return dataset

In [175]:
result = collection.query(query_texts="이 과제는 벵골어로 된 문장을 영어로 번역하는 것입니다.\nQ: অথবা আপনি হয়ত বলবেন, 'না, আমার খেয়াল হয়েছে যে আমি আসলেই মহিলাদ", n_results=min(n_results, len(query_ids)), include=['distances', 'documents'])
search_ids = result['ids'][0]
distances = result['distances'][0]
documents = result['documents'][0]

In [ ]:
distances, documents

In [177]:
def dedup_non_pair(dataset, data_format='sft'):
    def validate_non_pair(data):
        dedup_flag = False
        if data_format == 'sft':
            conversations = data['conversations']

            if len(conversations) == 0: # empty
                return False

            if conversations[0]["from"] != 'human': # skip first if it's not human
                conversations = conversations[1:]

            for idx, conv in enumerate(conversations): # check right pairs
                role = conv['from']
                if idx % 2 == 0 and role != 'human':
                    dedup_flag = True
                    break
                elif idx % 2 == 1 and role != 'gpt':
                    dedup_flag = True
                    break
        elif data_format == 'dpo':
            if data['input'] and data['chosen'] and data['rejected']:
                return True
            else:
                return False
                
        # if dedup_flag:
        #     print(conv)
        
        return not dedup_flag
    
    start = len(dataset)
    dataset = dataset.filter(validate_non_pair)
    print(f"{start - len(dataset)}/{start} deduped")
    return dataset

In [100]:
def dedup_too_much_token(dataset, data_format='sft', max_token=3800):
    api_server_url = "http://localhost:21122"
    def validate_too_much_token(data):
        dedup_flag = False
        if data_format == 'sft':
            convs = data['conversations']
        elif data_format == 'dpo':
            convs = [data['input'], data['chosen'], data['rejected']]
        else:
            print("data_format should be [sft, dpo]")
            return
        
        num_token = 0
        for conv in convs:
            if data_format == 'sft':
                _from = conv['from']
                _value = conv['value']
            else:
                _value = conv
            
            input_json = {
                "model_name": "DIE_10.7b_sft_v4_dpo_v2_ep3",
                "prompt": _value,
            }

            ret = requests.post(api_server_url + "/count_token", json=input_json)

            output_json = ret.json()
            num_token += output_json['count']
            
            if num_token > max_token:
                dedup_flag = True
                break

        # if dedup_flag:
        #     print(conv)
        
        return not dedup_flag
    
    start = len(dataset)
    dataset = dataset.filter(validate_too_much_token)
    print(f"{start - len(dataset)}/{start} deduped")
    return dataset

In [196]:
dataset_path = dpo_list3[2]
print(dataset_path)
dataset = load_dpo_dataset(dataset_path)
dataset

/data/llm_datasets/custom/kodpo/deduped/ko_ultrafeedback_binarized.json


Extracting data files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 861.08it/s]
Generating train split: 61807 examples [00:06, 9836.59 examples/s]


Dataset({
    features: ['input', 'chosen', 'task', 'id', 'rejected'],
    num_rows: 61807
})

In [195]:
dataset_ = dedup_non_pair(dataset, 'dpo')

Filter: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2418/2418 [00:00<00:00, 63124.85 examples/s]

0/2418 deduped


In [191]:
new_dataset = []
for data in dataset_:
    new_dataset.append(data)
    
with open('/data/llm_datasets/custom/kodpo/deduped/ko_ultrafeedback_binarized.json', "w") as json_file:
    json.dump(new_dataset, json_file)

In [222]:
""" find odd code blocks(dpo)"""
dataset_path = dpo_list4[2]
print(dataset_path)
# dataset = load_dataset("json", dataset_path)
dataset = load_dpo_dataset(dataset_path, split=None)
train_dataset = dataset['train']

# new_dataset = []
code_prefixes = []

odd_dataset = []
oddd_dataset = []
odd_idxs = set()
normal_dataset = []
flag_normal = True
flag_code = False
num_normal = 0
for idx, data in enumerate(train_dataset):
    # conversations = data['conversations']
    flag_normal = True
    for conv in [data['input'], data['chosen'], data['rejected']]:
        _value = conv
        flag_code = False
        find_iter = re.finditer('```', _value)
        temp_num = 0
        for fidx, ftext in enumerate(find_iter):
            flag_code = True
            start_index = ftext.start() + 3
            # new_dataset.append(data)
            #TODO: 스페이스바가 바로 오는 경우..
            candidate = re.split(r'[\n]', _value[start_index:])[0]
            if fidx % 2 == 0 and '```' not in candidate and candidate not in available_code_prefixes:
                odd_dataset.append((candidate, data))
                code_prefixes.append(candidate)
                odd_idxs.add(idx)
                flag_normal = False
                break
            temp_num += 1
        if temp_num % 2 != 0:
            oddd_dataset.append(('odd', data))
            odd_idxs.add(idx)
            flag_normal = False
        
        if not flag_normal:
            break
    
    if flag_code and flag_normal:
        normal_dataset.append(data)
        num_normal += 1

print(len(train_dataset), len(normal_dataset), len(odd_dataset), len(oddd_dataset), len(odd_idxs))

/data/llm_datasets/custom/kodpo/translated/ko_distilabel-math-preference-dpo.json
2418 3 2 0 2


In [225]:
odd_dataset[1]

(' ',
 {'input': '행렬과 가우시안 제거를 사용하여 선형 방정식 시스템을 푸는 절차를 자세히 설명하세요.\n 심호흡을 하고 단계별로 천천히 생각한 후 정확한 답변을 제공하세요.',
  'chosen': '행렬과 가우시안 제거를 사용하여 선형 방정식 문제를 풀려면 다음 단계를 따르세요:1. 주어진 선형 방정식 시스템을 행렬 형식으로 적습니다. 예를 들어, 다음과 같은 방정식이 주어졌다고 가정합니다:   - x + 2y - z = 4   - 2x - y + z = 1   - 3x + y + z = 2   행렬 표현은 다음과 같습니다:   [1  2 -1 | 4]   [2 -1 | 1]   [3 | 1] 1 | 2]2. 가우시안 제거를 적용하여 행렬을 행렬의 대각선 형태로 변환합니다. 주요 목표는 행렬의 왼쪽 아래 부분이 모두 0이 되도록 하는 것입니다. 왼쪽 열(열 1)에 초점을 맞추고 행렬을 행으로 나누어 대각선 요소 아래에 0이 되도록 행렬을 행으로 나눕니다.   - 행렬에 왼쪽 위쪽 모서리에 0이 아닌 요소가 있는 행이 있으면 그 행을 같은 열에 0이 아닌 요소가 있는 행과 바꾸세요. 이렇게 하면 나중에 0으로 나누는 것을 피할 수 있습니다.   - 행을 대각선 값으로 나누어 행을 1로 만듭니다.   행을 행으로 나누어 행을 행으로 나눕니다. 3. 행렬이 행렬 대각선 형식이 되면 행에서 아래로 내려가서 주요 대각선(역수법)에서 0이 아닌 요소를 제거합니다. 오른쪽 열(아직 0이 아닌 경우)을 먼저 시작하고 행 곱셈을 사용하여 피벗 위에 있는 요소를 0으로 만듭니다. 대각선 위에 있는 모든 0이 아닌 요소를 제거할 때까지 열별로 아래로 내려가면서 작업합니다.4. 결과 행렬은 행렬 대각선 형식이 됩니다. 오른쪽 열은 방정식 시스템의 변수에 대한 해를 나타냅니다.   - 행렬에 해가 없거나 무한한 해가 있으면 행렬 대각선 형식에서 분명해질 것입니다.   - 각 변수에 고유한 해가 있으면 행렬은 해로 구성된 행렬의 형태로 변합니다.1단계의 예를 사용하면

In [230]:
new_dataset = []
for data in combined_dataset:
    new_dataset.append(data)
    
with open('/data/llm_datasets/custom/ados/dpo/ados_dpo_v2.json', "w") as json_file:
    json.dump(new_dataset, json_file)

In [228]:
# from fastchat.train.data_modules.sft_dataset import load_sft_dataset, combine_dataset
combined_dataset = concatenate_datasets([load_dpo_dataset(dataset_path) for dataset_path in dpo_list5])
combined_dataset.features

Extracting data files: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1120.87it/s]
Generating train split: 61418 examples [00:06, 10150.79 examples/s]


{'input': Value(dtype='string', id=None),
 'chosen': Value(dtype='string', id=None),
 'task': Value(dtype='string', id=None),
 'id': Value(dtype='string', id=None),
 'rejected': Value(dtype='string', id=None),
 'system': Value(dtype='string', id=None)}

In [237]:
combined_dataset[70002]

{'input': '백악관 - 도널드 트럼프 대통령은 월요일 자신에게 "절대적인 권리"가 있으며 2016년 대선 캠페인와 러시아와의 관련성에 대한 연방 정부의 조사를 "완전히 위헌적"이라고 주장하면서 트위터에 트윗을 올렸습니다. 트럼프는 기존 규정을 무시하고 대통령의 권한에 대한 이해된 한계를 무시하는 경우가 많지만, 이번 발언은 일부 법률 학자들에 의해 전례가 없는 것으로 간주되며 헌법의 한계를 시험하고 있습니다. 조지타운 대학의 법학 교수인 수잔 로브 블러치는 "트럼프의 트윗에 충격을 받았다"고 말하며 "대통령은 자신을 사면할 수 없습니다. 법원이 "아니요"라고 말했고, 대통령이 사면을 주면 "대부분의 의회는 탄핵할 것"이라고 말했다. 이 교수는 대통령이 재임 중 기소되고 재판을 받을 수 있는지에 대해 상원에서 증언한 바 있는 교수로, 특검은 "완전히 헌법에 부합한다"고 말합니다. 프린스턴 대학의 다른 헌법학자인 키스 휘트링턴 교수는 "대통령의 사면에 관한 심각한 논쟁이 법학자들 사이에서 논의되고 있는 점을 언급한 것은 중요하지만, 이는 궁극적으로 법원에서 해결될 어려운 특이한 헌법적 질문이라고 생각합니다."라고 말했습니다. 노스웨스턴 대학교 법대 교수 앤드류 코펠먼에 따르면, "사면에 관한 트럼프의 입장은 헌법의 텍스트에 부합하지만 전반적인 구조와 모순됩니다. 대통령이 사면을 통해 사형이나 다른 연방 범죄를 저지른 후 즉시 자신을 사면할 수 있다는 의미입니다."라고 말했습니다. "헌법 전체에 맞지 않습니다." 트럼프가 자신에 대한 사면 가능성에 대해 기자에게 질문하자 백악관 대변인 사라 샌더스 대변인은 "질문: 어느 대통령이 자신을 사면하려 한 적이 없다고 VOA에 말했나요? 옵션: A. 조지타운 대학의 수잔 로브 블러치 B. 백악관 대변인 사라 샌더스 C. 프린스턴 대학의 키트 휘트링 D. 정보가 충분하지 않음 === 정답은',
 'chosen': 'A. 조지 워싱턴 대학교의 수잔 로우 블로치',
 'task': 'dpo_system',
 'id': '

In [12]:
class ados_DPODataset:
    def __init__(
        self, 
        dataset_path="/data/llm_datasets/custom/ados/dpo/ados_dpo_v2.json",
        eval_dataset_path = "", #/data/llm_datasets/Ultrafeedback_binarized.ko.hankang/test_prefs.json.kr
        data_format='chat-orca',
        # search_term='\n\n### Assistant:',
        num_train=None,
        num_eval=None,
    ):
        self.dataset_path = dataset_path
        self.eval_dataset_path = dataset_path
        if eval_dataset_path:
            self.eval_dataset_path = eval_dataset_path
        self.data_format = data_format
        # self.search_term = search_term
        self.num_train = num_train
        self.num_eval = num_eval
    
    def get_prompt_and_response(self, data):
        conv = get_conversation_template(self.data_format)
        if data['system']:
            conv.system_message = conv.tasks['system_instruct'].format(system=data['system'])
        conv.append_message(conv.roles[0], data['input'])
        conv.append_message(conv.roles[1], '')
        prompt = conv.get_prompt()
        conv.update_last_message(data['chosen'])
        chosen = conv.get_prompt()[len(prompt):]
        conv.update_last_message(data['rejected'])
        rejected = conv.get_prompt()[len(prompt):]
        
        return prompt, chosen, rejected
    
    def make_dpo_data_module(self):
        def split_prompt_and_responses(data) -> Dict[str, str]:
            prompt, chosen, rejected = self.get_prompt_and_response(data)
            # prompt = extract_anthropic_prompt(prompt_and_response, self.search_term)
            # promopt_rejected = extract_anthropic_prompt(prompt_and_response_rejected, self.search_term)
            return {
                "prompt": prompt,
                "chosen": chosen,
                "rejected": rejected,
            }
                             
        train_dataset = load_dpo_dataset(self.dataset_path)
        eval_dataset = load_dpo_dataset(self.eval_dataset_path)

        original_columns = list(train_dataset.features.keys())
        original_columns_eval = list(eval_dataset.features.keys())

        if self.num_train is not None:
            train_dataset = train_dataset.select(range(min(len(train_dataset), self.num_train)))
        if self.num_eval is not None:
            eval_dataset = eval_dataset.select(range(min(len(train_dataset), self.num_eval)))

        train_dataset = train_dataset.map(split_prompt_and_responses, remove_columns=original_columns)
        eval_dataset = eval_dataset.map(split_prompt_and_responses, remove_columns=original_columns_eval)

        return dict(train_dataset=train_dataset, eval_dataset=eval_dataset)

In [4]:
from fastchat.train.data_modules.dpo_dataset import ados_DPODataset

dpo_dataset = ados_DPODataset()
dpo_datamodule = dpo_dataset.make_dpo_data_module()

In [5]:
dpo_datamodule['train_dataset'][-5000]

{'chosen': ' 질문은 다음과 같을 수 있습니다: "필립 바드의 연구에 따르면 감각 정보와 운동 정보 외에 어떤 유형의 정보가 더 이상 처리되기 전에 중뇌를 통과해야 하는 정보에는 어떤 것이 있었습니까?"</s>',
 'rejected': '  이 글에서 제공된 정보에 따르면, 감각, 운동, 생리적 정보 모두 추가 처리를 받기 전에 감각계(특히 시상)를 통과해야 했습니다. 따라서 질문에 대한 정답은 다음과 같습니다:\n생리적 정보.\n\n카논은 감각적 사건이 의식을 촉발하기 전에 생리적 반응을 유발할 수 있는 것은 해부학적으로 불가능하며, 감정적 자극은 생리적 반응과 경험적 측면의 감정을 동시에 유발해야 한다고 주장했기 때문입니다.</s>',
 'prompt': '### System:\nYou are an AI assistant. You will be given a task. You must generate a detailed and long answer.\n\n### User: 이 기사에 대한 질문은 무엇인가요? 질문이 불가능한 경우 "불가능"이라고 말하세요.\n\n필립 바드는 동물 연구를 통해 이론에 기여했습니다. 바드는 감각, 운동 및 생리적 정보가 더 이상의 처리를 받기 전에 모두 시상(특히 시상)을 통과해야 한다는 것을 발견했습니다. 따라서 카논은 감각적 이벤트가 의식적 인식을 촉발하기 전에 생리적 반응을 유발할 수 없으며, 감정적 자극이 생리적 및 경험적 측면의 감정을 동시에 유발해야 한다고 주장했습니다.\n\n 감각 및 운동 정보와 함께 처리된 후 시상에는 어떤 정보가 통과해야 했나요?\n\n### Assistant:'}

In [259]:
data = combined_dataset[70002]

conv = get_conversation_template('chat-orca')
conv.system_message = conv.tasks['system_instruct'].format(system=data['system'])
conv.append_message(conv.roles[0], data['input'])
conv.append_message(conv.roles[1], '')
prompt = conv.get_prompt()

conv.update_last_message(data['chosen'])
chosen = conv.get_prompt()[len(prompt):]
conv.update_last_message(data['rejected'])
rejected = conv.get_prompt()[len(prompt):]

In [179]:
from transformers import AutoTokenizer
# model_path = "/workspaces/disk0/data/llm_weights/Platypus2-70B-instruct/"
# model_path = "/workspaces/disk0/data/llm_weights/vicuna-13b-v1.5/"
model_path = "/workspaces/disk0/data/llm_weights/MoMo-70B-lora-1.8.4-DPO/"
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    model_max_length=4096,
    padding_side="right",
    use_fast=False,
)
tokenizer.pad_token = tokenizer.unk_token

In [ ]:
tokenizer

In [4]:
raw_data = load_sft_dataset("/data/llm_datasets/custom/ados/sft/ados_sft_v4.1.json")

In [34]:
raw_data[50000]

{'system': None,
 'conversations': [{'from': 'human', 'value': '아케치_미쓰히데에 대해서 알고 있니, <|bot|>?'},
  {'from': 'gpt', 'value': '네, 글의 내용을 전부 이해하였습니다.'},
  {'from': 'human',
   'value': '그렇다면, 이에야스와 히데요시에 대한 여러 가지 이야기를 들어봤는데, 그들의 실제 인물이 어떤지 설명해줄 수 있을까요?'},
  {'from': 'gpt',
   'value': '이에야스의 경우, 죽인 원한으로 인해 미쓰히데의 친척인 후쿠를 천거하였고, 히데요시는 사쿠마 노부모리, 하야시 히데사다 등의 추방으로 불안에 떨었으며, 모리와 전쟁 중(빗추 다카마쓰 성 공격)이었지만 모리와 강화를 맺고 군사를 교토로 되돌린 이후 히데요시가 최대 수혜자가 되었습니다.'},
  {'from': 'human', 'value': '그렇군요. 이에 대한 다른 의견도 있을까요? '},
  {'from': 'gpt',
   'value': '네, 호소카와 후지타카와 오다 노부타다가 흑막이라는 설이 있습니다. 하지만 이것은 대다수의 학자들이 부정하는 설입니다.'},
  {'from': 'human',
   'value': '그렇군요. 이번 글에서는 두 사람 외의 인물도 등장했는데, 그들에 대한 자세한 설명도 있을까요? '},
  {'from': 'gpt',
   'value': '아케치_미쓰히데에서는 도쿠가와 이에미쓰와 사이토 도시미쓰의 딸 후쿠(가스가노 쓰보네(春日局)), 사쿠마 노부모리, 하야시 히데사다 등 여러 사람들도 언급되었습니다. 하지만 이들에 대한 자세한 정보는 별도의 글에서 찾아보는 것이 좋을 것 같습니다.'}],
 'task_name': 'instruct',
 'task': None,
 'instruction': '오다가를 둘러싼 여러 장수들의 흑막설. 도쿠가와 이에야스와 도요토미 히데요시가 주로 거론된다. 이에야스의 경우. 노부나가의 명령으

In [8]:
from fastchat.train.train import LazySupervisedDataset
dataset = LazySupervisedDataset(raw_data, tokenizer, data_format="vicuna")

In [250]:
sources = [raw_data[50000]["conversations"]]
data_format="qwen"
custom_system_message=None
conv = get_conversation_template(data_format)
roles = {"human": conv.roles[0], "gpt": conv.roles[1]}

if custom_system_message is not None:
    conv.system_message = custom_system_message

# Apply prompt templates
conversations = []
for i, source in enumerate(sources):
    if roles[source[0]["from"]] != conv.roles[0]:
        # Skip the first one if it is not from human
        source = source[1:]

    conv.messages = []
    for j, sentence in enumerate(source):
        role = roles[sentence["from"]]
        assert role == conv.roles[j % 2], f"{i}"
        conv.append_message(role, sentence["value"])
    conversations.append(conv.get_prompt())

# Tokenize conversations
input_ids = tokenizer(
    conversations,
    return_tensors="pt",
    padding="max_length", #memo TODO False=GPTNeox max_length=Llama
    # padding=False,
    max_length=tokenizer.model_max_length,
    truncation=True,
).input_ids
targets = input_ids.clone()

In [251]:
sep = conv.sep + "\n" + conv.roles[1] + "\n"
sep2 = conv.sep + "\n" + conv.roles[0] + "\n"
len_im_end_nl = len(tokenizer(conv.sep + "\n"))
len_sep2 = len(tokenizer(sep2))
len_im_start_nl = len(tokenizer(conv.roles[0] + "\n"))
for conversation, target in zip(conversations, targets):
    total_len = int(target.ne(tokenizer.pad_token_id).sum())

    turns = conversation.split(sep2)
    cur_len = len(tokenizer(turns[0] + sep2).input_ids) # first turn is system
    target[:cur_len] = IGNORE_TOKEN_ID
    for i, turn in enumerate(turns[1:]):
        if turn == "":
            break
        turn_len = len(tokenizer(turn).input_ids)

        parts = turn.split(sep)
        if len(parts) != 2:
            break
        parts[0] += sep
        # "-2" is hardcoded for the LLaMA tokenizer to make the offset correct.
        instruction_len = len(tokenizer(parts[0]).input_ids)

        # Ignore the user instructions
        target[cur_len : cur_len + instruction_len] = IGNORE_TOKEN_ID
        cur_len += turn_len
        target[cur_len + len_im_end_nl:cur_len+len_sep2] = IGNORE_TOKEN_ID
        cur_len += len_sep2

    target[cur_len:] = IGNORE_TOKEN_ID
    
    if cur_len < tokenizer.model_max_length:
        if cur_len != total_len:
    #         target[:] = IGNORE_TOKEN_ID
            print(
                f"WARNING: tokenization mismatch: {cur_len} vs. {total_len}."
                f" (ignored)"
            )

In [243]:
sep2

In [270]:
sources = [raw_data[50000]["conversations"]]
data_format="qwen"
custom_system_message="hello."

conv = get_conversation_template(data_format)
roles = {"human": conv.roles[0], "gpt": conv.roles[1]}

if custom_system_message is not None:
    conv.system_message = custom_system_message

# Apply prompt templates for qwen
if data_format == "qwen":
    assert tokenizer("<|im_end|>").input_ids[0] == 151645

    max_len = tokenizer.model_max_length
    im_start = 151644
    im_end = 151645
    nl_tokens = tokenizer('\n').input_ids
    _system = tokenizer('system').input_ids + nl_tokens
    _user = tokenizer('user').input_ids + nl_tokens
    _assistant = tokenizer('assistant').input_ids + nl_tokens

    # Apply prompt templates
    input_ids, targets = [], []
    for i, source in enumerate(sources):
        if roles[source[0]["from"]] != roles["human"]:
            source = source[1:]

        input_id, target = [], []
        system = [im_start] + _system + tokenizer(conv.system_message).input_ids + [im_end] + nl_tokens
        input_id += system
        target += [im_start] + [IGNORE_TOKEN_ID] * (len(system)-3) + [im_end] + nl_tokens
        assert len(input_id) == len(target)
        for j, sentence in enumerate(source):
            role = roles[sentence["from"]]
            _input_id = tokenizer(role).input_ids + nl_tokens + \
                tokenizer(sentence["value"]).input_ids + [im_end] + nl_tokens
            input_id += _input_id
            if role == '<|im_start|>user':
                _target = [im_start] + [IGNORE_TOKEN_ID] * (len(_input_id)-3) + [im_end] + nl_tokens
            elif role == '<|im_start|>assistant':
                _target = [im_start] + [IGNORE_TOKEN_ID] * len(tokenizer(role).input_ids) + \
                    _input_id[len(tokenizer(role).input_ids)+1:-2] + [im_end] + nl_tokens
            else:
                raise NotImplementedError
            target += _target
        assert len(input_id) == len(target)
        input_id += [tokenizer.pad_token_id] * (max_len - len(input_id))
        target += [IGNORE_TOKEN_ID] * (max_len - len(target))
        input_ids.append(input_id[:max_len])
        targets.append(target[:max_len])
    input_ids = torch.tensor(input_ids, dtype=torch.int)
    targets = torch.tensor(targets, dtype=torch.int)

In [168]:
turns

['<|im_start|>system\nYou are a helpful assistant.',
 '아케치_미쓰히데에 대해서 알고 있니, <|bot|>?<|im_end|>\n<|im_start|>assistant\n네, 글의 내용을 전부 이해하였습니다.',
 '그렇다면, 이에야스와 히데요시에 대한 여러 가지 이야기를 들어봤는데, 그들의 실제 인물이 어떤지 설명해줄 수 있을까요?<|im_end|>\n<|im_start|>assistant\n이에야스의 경우, 죽인 원한으로 인해 미쓰히데의 친척인 후쿠를 천거하였고, 히데요시는 사쿠마 노부모리, 하야시 히데사다 등의 추방으로 불안에 떨었으며, 모리와 전쟁 중(빗추 다카마쓰 성 공격)이었지만 모리와 강화를 맺고 군사를 교토로 되돌린 이후 히데요시가 최대 수혜자가 되었습니다.',
 '그렇군요. 이에 대한 다른 의견도 있을까요? <|im_end|>\n<|im_start|>assistant\n네, 호소카와 후지타카와 오다 노부타다가 흑막이라는 설이 있습니다. 하지만 이것은 대다수의 학자들이 부정하는 설입니다.',
 '그렇군요. 이번 글에서는 두 사람 외의 인물도 등장했는데, 그들에 대한 자세한 설명도 있을까요? <|im_end|>\n<|im_start|>assistant\n아케치_미쓰히데에서는 도쿠가와 이에미쓰와 사이토 도시미쓰의 딸 후쿠(가스가노 쓰보네(春日局)), 사쿠마 노부모리, 하야시 히데사다 등 여러 사람들도 언급되었습니다. 하지만 이들에 대한 자세한 정보는 별도의 글에서 찾아보는 것이 좋을 것 같습니다.<|im_end|>\n']

In [266]:
tokenizer("<|im_end|>").input_ids[0] == 151645

True

In [165]:
sep = conv.sep + "\n" + conv.roles[1] + "\n"
sep

'<|im_end|>\n<|im_start|>assistant\n'

In [181]:
conversations

['<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n아케치_미쓰히데에 대해서 알고 있니, <|bot|>?<|im_end|>\n<|im_start|>assistant\n네, 글의 내용을 전부 이해하였습니다.<|im_end|>\n<|im_start|>user\n그렇다면, 이에야스와 히데요시에 대한 여러 가지 이야기를 들어봤는데, 그들의 실제 인물이 어떤지 설명해줄 수 있을까요?<|im_end|>\n<|im_start|>assistant\n이에야스의 경우, 죽인 원한으로 인해 미쓰히데의 친척인 후쿠를 천거하였고, 히데요시는 사쿠마 노부모리, 하야시 히데사다 등의 추방으로 불안에 떨었으며, 모리와 전쟁 중(빗추 다카마쓰 성 공격)이었지만 모리와 강화를 맺고 군사를 교토로 되돌린 이후 히데요시가 최대 수혜자가 되었습니다.<|im_end|>\n<|im_start|>user\n그렇군요. 이에 대한 다른 의견도 있을까요? <|im_end|>\n<|im_start|>assistant\n네, 호소카와 후지타카와 오다 노부타다가 흑막이라는 설이 있습니다. 하지만 이것은 대다수의 학자들이 부정하는 설입니다.<|im_end|>\n<|im_start|>user\n그렇군요. 이번 글에서는 두 사람 외의 인물도 등장했는데, 그들에 대한 자세한 설명도 있을까요? <|im_end|>\n<|im_start|>assistant\n아케치_미쓰히데에서는 도쿠가와 이에미쓰와 사이토 도시미쓰의 딸 후쿠(가스가노 쓰보네(春日局)), 사쿠마 노부모리, 하야시 히데사다 등 여러 사람들도 언급되었습니다. 하지만 이들에 대한 자세한 정보는 별도의 글에서 찾아보는 것이 좋을 것 같습니다.<|im_end|>\n']

In [113]:
conversations

['### System:\nYou are an AI assistant, please behave and help the user.\n\n### User: 아케치_미쓰히데에 대해서 알고 있니, <|bot|>?\n\n### Assistant: 네, 글의 내용을 전부 이해하였습니다.</s>### User: 그렇다면, 이에야스와 히데요시에 대한 여러 가지 이야기를 들어봤는데, 그들의 실제 인물이 어떤지 설명해줄 수 있을까요?\n\n### Assistant: 이에야스의 경우, 죽인 원한으로 인해 미쓰히데의 친척인 후쿠를 천거하였고, 히데요시는 사쿠마 노부모리, 하야시 히데사다 등의 추방으로 불안에 떨었으며, 모리와 전쟁 중(빗추 다카마쓰 성 공격)이었지만 모리와 강화를 맺고 군사를 교토로 되돌린 이후 히데요시가 최대 수혜자가 되었습니다.</s>### User: 그렇군요. 이에 대한 다른 의견도 있을까요? \n\n### Assistant: 네, 호소카와 후지타카와 오다 노부타다가 흑막이라는 설이 있습니다. 하지만 이것은 대다수의 학자들이 부정하는 설입니다.</s>### User: 그렇군요. 이번 글에서는 두 사람 외의 인물도 등장했는데, 그들에 대한 자세한 설명도 있을까요? \n\n### Assistant: 아케치_미쓰히데에서는 도쿠가와 이에미쓰와 사이토 도시미쓰의 딸 후쿠(가스가노 쓰보네(春日局)), 사쿠마 노부모리, 하야시 히데사다 등 여러 사람들도 언급되었습니다. 하지만 이들에 대한 자세한 정보는 별도의 글에서 찾아보는 것이 좋을 것 같습니다.</s>']

In [38]:
conversations

["A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. USER: 아케치_미쓰히데에 대해서 알고 있니, <|bot|>? ASSISTANT: 네, 글의 내용을 전부 이해하였습니다.</s>USER: 그렇다면, 이에야스와 히데요시에 대한 여러 가지 이야기를 들어봤는데, 그들의 실제 인물이 어떤지 설명해줄 수 있을까요? ASSISTANT: 이에야스의 경우, 죽인 원한으로 인해 미쓰히데의 친척인 후쿠를 천거하였고, 히데요시는 사쿠마 노부모리, 하야시 히데사다 등의 추방으로 불안에 떨었으며, 모리와 전쟁 중(빗추 다카마쓰 성 공격)이었지만 모리와 강화를 맺고 군사를 교토로 되돌린 이후 히데요시가 최대 수혜자가 되었습니다.</s>USER: 그렇군요. 이에 대한 다른 의견도 있을까요?  ASSISTANT: 네, 호소카와 후지타카와 오다 노부타다가 흑막이라는 설이 있습니다. 하지만 이것은 대다수의 학자들이 부정하는 설입니다.</s>USER: 그렇군요. 이번 글에서는 두 사람 외의 인물도 등장했는데, 그들에 대한 자세한 설명도 있을까요?  ASSISTANT: 아케치_미쓰히데에서는 도쿠가와 이에미쓰와 사이토 도시미쓰의 딸 후쿠(가스가노 쓰보네(春日局)), 사쿠마 노부모리, 하야시 히데사다 등 여러 사람들도 언급되었습니다. 하지만 이들에 대한 자세한 정보는 별도의 글에서 찾아보는 것이 좋을 것 같습니다.</s>"]

In [184]:
conv.get_prompt()

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n아케치_미쓰히데에 대해서 알고 있니, <|bot|>?<|im_end|>\n<|im_start|>assistant\n네, 글의 내용을 전부 이해하였습니다.<|im_end|>\n<|im_start|>user\n그렇다면, 이에야스와 히데요시에 대한 여러 가지 이야기를 들어봤는데, 그들의 실제 인물이 어떤지 설명해줄 수 있을까요?<|im_end|>\n<|im_start|>assistant\n이에야스의 경우, 죽인 원한으로 인해 미쓰히데의 친척인 후쿠를 천거하였고, 히데요시는 사쿠마 노부모리, 하야시 히데사다 등의 추방으로 불안에 떨었으며, 모리와 전쟁 중(빗추 다카마쓰 성 공격)이었지만 모리와 강화를 맺고 군사를 교토로 되돌린 이후 히데요시가 최대 수혜자가 되었습니다.<|im_end|>\n<|im_start|>user\n그렇군요. 이에 대한 다른 의견도 있을까요? <|im_end|>\n<|im_start|>assistant\n네, 호소카와 후지타카와 오다 노부타다가 흑막이라는 설이 있습니다. 하지만 이것은 대다수의 학자들이 부정하는 설입니다.<|im_end|>\n<|im_start|>user\n그렇군요. 이번 글에서는 두 사람 외의 인물도 등장했는데, 그들에 대한 자세한 설명도 있을까요? <|im_end|>\n<|im_start|>assistant\n아케치_미쓰히데에서는 도쿠가와 이에미쓰와 사이토 도시미쓰의 딸 후쿠(가스가노 쓰보네(春日局)), 사쿠마 노부모리, 하야시 히데사다 등 여러 사람들도 언급되었습니다. 하지만 이들에 대한 자세한 정보는 별도의 글에서 찾아보는 것이 좋을 것 같습니다.<|im_end|>\n'

In [12]:
from transformers.trainer_pt_utils import LabelSmoother
import torch
IGNORE_TOKEN_ID = LabelSmoother.ignore_index

In [9]:
dataset[50000]

{'input_ids': tensor([    1,   319, 13563,  ...,     0,     0,     0]),
 'labels': tensor([-100, -100, -100,  ..., -100, -100, -100]),
 'attention_mask': tensor([ True,  True,  True,  ..., False, False, False])}

In [55]:
targets.shape

torch.Size([1, 4096])

In [271]:
tokenizer.decode(input_ids[0])

'<|im_start|> system\nhello. <|im_end|> \n <|im_start|> user\n아케치_미쓰히데에 대해서 알고 있니, <|bot|>? <|im_end|> \n <|im_start|> assistant\n네, 글의 내용을 전부 이해하였습니다. <|im_end|> \n <|im_start|> user\n그렇다면, 이에야스와 히데요시에 대한 여러 가지 이야기를 들어봤는데, 그들의 실제 인물이 어떤지 설명해줄 수 있을까요? <|im_end|> \n <|im_start|> assistant\n이에야스의 경우, 죽인 원한으로 인해 미쓰히데의 친척인 후쿠를 천거하였고, 히데요시는 사쿠마 노부모리, 하야시 히데사다 등의 추방으로 불안에 떨었으며, 모리와 전쟁 중(빗추 다카마쓰 성 공격)이었지만 모리와 강화를 맺고 군사를 교토로 되돌린 이후 히데요시가 최대 수혜자가 되었습니다. <|im_end|> \n <|im_start|> user\n그렇군요. 이에 대한 다른 의견도 있을까요?  <|im_end|> \n <|im_start|> assistant\n네, 호소카와 후지타카와 오다 노부타다가 흑막이라는 설이 있습니다. 하지만 이것은 대다수의 학자들이 부정하는 설입니다. <|im_end|> \n <|im_start|> user\n그렇군요. 이번 글에서는 두 사람 외의 인물도 등장했는데, 그들에 대한 자세한 설명도 있을까요?  <|im_end|> \n <|im_start|> assistant\n아케치_미쓰히데에서는 도쿠가와 이에미쓰와 사이토 도시미쓰의 딸 후쿠(가스가노 쓰보네(春日局)), 사쿠마 노부모리, 하야시 히데사다 등 여러 사람들도 언급되었습니다. 하지만 이들에 대한 자세한 정보는 별도의 글에서 찾아보는 것이 좋을 것 같습니다. <|im_end|> \n<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endo

In [272]:
tokenizer.decode(torch.where(targets[0] == IGNORE_TOKEN_ID, tokenizer.unk_token_id, targets[0]))

'<|im_start|> <|endoftext|><|endoftext|><|endoftext|><|endoftext|> <|im_end|> \n <|im_start|> <|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|> <|im_end|> \n <|im_start|> <|endoftext|><|endoftext|>네, 글의 내용을 전부 이해하였습니다. <|im_end|> \n <|im_start|> <|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftex

In [ ]:
# sep = ""
# sep2 = ?
# role1 = "<|im_start|>assistant"
# add_chatml = "\n"
ret = "" if system_prompt == "" else system_prompt + self.sep + "\n"
for role, message in self.messages:
    if message:
        ret += role + "\n" + message + self.sep + "\n"
    else:
        ret += role + "\n"
sep + role1 + add_chatml
"" + "<|im_start|>user" + "\n"

In [ ]:
# sep="\n\n",
# role1 = "### Assistant"
# sep2="</s>",
# add_colon = ": "
seps = [self.sep, self.sep2]
ret = system_prompt + seps[0]
for i, (role, message) in enumerate(self.messages):
    if message:
        ret += role + ": " + message + seps[i % 2]
    else:
        ret += role + ":"
sep + role1 + add_colon
"\n\n" + "### Assistant" + ": "

In [ ]:
sep=" ",
sep2="</s>",

In [3]:
dpo_dataset = ados_DPODataset("/data/llm_datasets/custom/ados/dpo/ados_mdpo_v2.json")

NameError: name 'ados_DPODataset' is not defined

In [119]:
def time_to_seconds(time_str):
    # 시간 문자열을 시, 분, 초로 분리
    parts = time_str.split("시")[1].split("분")[0].split("초")
    # 시, 분, 초를 숫자로 변환
    hours, minutes, seconds = map(int, parts)
    # 초 단위 숫자로 변환
    return hours * 3600 + minutes * 60 + seconds

# 사용 예제
time_str = "2시 30분 15초"
seconds = time_to_seconds(time_str)
print(f"{time_str}은(는) {seconds}초와 같습니다.")

ValueError: not enough values to unpack (expected 3, got 1)